In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
from matplotlib import style
import matplotlib.pyplot as plt
from sklearn import datasets
from jupyterthemes import jtplot
import json

from datetime import datetime

In [2]:
CitiPath = 'Resources/2022/202203-citibike-tripdata.csv'
citi_2203_df = pd.read_csv(CitiPath)
citi_2203_df.head()

/Users/ngocdiep/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,FA2F660C8D433037,electric_bike,2022-03-04 18:58:36,2022-03-04 19:13:51,Washington St & Barrow St,5847.08,University Pl & E 14 St,5905.14,40.731911,-74.008769,40.734814,-73.992085,member
1,03B6FE6FAEE61465,classic_bike,2022-03-06 20:37:09,2022-03-06 20:45:28,Washington St & Barrow St,5847.08,W 11 St & 6 Ave,5914.03,40.731911,-74.008769,40.735324,-73.998004,member
2,6E6995761B4A5760,classic_bike,2022-03-01 15:13:49,2022-03-01 15:19:53,1 Ave & E 110 St,7522.02,E 110 St & Madison Ave,7587.14,40.792327,-73.938300,40.796154,-73.947821,member
3,AA51CCF6CEDD6033,classic_bike,2022-03-11 13:59:03,2022-03-11 14:06:43,Crescent St & Broadway,6827.11,31 Rd & 21 St,6929.02,40.763359,-73.928647,40.766824,-73.930486,member
4,72AA3A7D430D8EF8,classic_bike,2022-03-08 14:59:16,2022-03-08 15:05:04,Crescent St & Broadway,6827.11,31 Rd & 21 St,6929.02,40.763359,-73.928647,40.766824,-73.930486,member


In [3]:
# print total of trips have been recorded in chosen period (Mar 2022)

print(f" there are totally {len(citi_2203_df)} in the chosen period (Mar_2022)")

 there are totally 1893444 in the chosen period (Mar_2022)


In [4]:
# find how many type of membership
citi_2203_df['member_casual'].unique()


array(['member', 'casual'], dtype=object)

In [5]:
number_station = citi_2203_df['start_station_name'].unique()
number_station

array(['Washington St & Barrow St', '1 Ave & E 110 St',
       'Crescent St & Broadway', ..., '63 St & 5 Ave', nan,
       'Morgan WH station'], dtype=object)

In [6]:
len(number_station)

1544

In [20]:
null_station = citi_2203_df.loc[citi_2203_df["start_station_name"] == "W 21 St & 6 Ave", ["ride_id", "start_station_name", "rideable_type"]]
null_station.head(10)
len(null_station)

8577

In [23]:
# top 10
extract_count = citi_2203_df["start_station_name"].value_counts()
extract_count.head(10)

top_3_station = extract_count.nlargest(3)
top_3_station

W 21 St & 6 Ave       8577
1 Ave & E 68 St       8135
Broadway & W 58 St    7735
Name: start_station_name, dtype: int64

In [24]:
bottom_3_station = extract_count.nsmallest(3)
bottom_3_station

20 Ave & 31 St             2
Morgan WH station          2
Boerum Pl\t& Pacific St    5
Name: start_station_name, dtype: int64

In [9]:
# extract_count.sort_values([0], axis=0).groupby([0]).tail(10)

In [10]:
# citi_2203_df = citi_2203_df.add_column(extract_count)

In [11]:
# find out how many trips made by members:

members =len(citi_2203_df.loc[citi_2203_df['member_casual']=='member'])

# How many trips made by casual members

casual = len(citi_2203_df.loc[citi_2203_df['member_casual']=='casual'])
#print(f"there are {len(citi_2203_df.loc[citi_2203_df["member_casual"]=="casual")}")

In [12]:
print(f" there are {members} trips by members")
print(f"there are {casual} trips by casual")

 there are 1514163 trips by members
there are 379281 trips by casual


In [18]:
# casual percentage:
percent_casual = casual/len(citi_2203_df)*100
percent_casual
percent_members = members/len(citi_2203_df)*100
percent_members 

79.96872365911007

In [14]:
#total trips in Mar 2022

total_trips = members + casual
total_trips

1893444

In [15]:
citi_2203_df['rideable_type'].unique()

array(['electric_bike', 'classic_bike', 'docked_bike'], dtype=object)

In [29]:
electric_trip = len(citi_2203_df.loc[citi_2203_df['rideable_type']=='electric_bike'])
classic_trip = len(citi_2203_df.loc[citi_2203_df['rideable_type']=='classic_bike'])
docked_trip = len(citi_2203_df.loc[citi_2203_df['rideable_type']=='docked_bike'])
print(f" electric trips: {electric_trip}\
      classic trips: {classic_trip}\
          docked trip: {docked_trip}")

 electric trips: 779271      classic trips: 1100249          docked trip: 13924


In [31]:
electric_percent = electric_trip/len(citi_2203_df)*100
classic_percent = classic_trip/len(citi_2203_df)*100
docked_percent = docked_trip/len(citi_2203_df)*100
print(f" electric: {electric_percent}\n"
      f"classic: {classic_percent}\n"
      f"docked: {docked_percent}")

 electric: 41.15627396426829
classic: 58.108346483973115
docked: 0.7353795517585944


In [ ]:
# try to join the above 2 df together (same columns, connecting rows)

# concate_df = [citi_2201_df, citi_2202_df,citi_2203_df]

# three_month_df = pd.concat(concate_df)
# three_month_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,55262E4365A955A2,classic_bike,2022-01-18 08:23:52,2022-01-18 08:28:18,Boerum Pl\t& Pacific St,4488.09,Clinton St & Joralemon St,4605.04,40.688489,-73.991160,40.692395,-73.993379,member
1,D272F1B15D841EC0,classic_bike,2022-01-21 09:03:22,2022-01-21 09:05:44,E 12 St & Ave C,5616.08,E 10 St & Avenue A,5659.05,40.727243,-73.976831,40.727408,-73.981420,member
2,D1FCEF55EB4A807F,classic_bike,2022-01-22 14:28:32,2022-01-22 14:53:18,W 21 St & 6 Ave,6140.05,W 44 St & 11 Ave,6756.05,40.741740,-73.994156,40.762009,-73.996975,member
3,E9CBDC6A0162C068,electric_bike,2022-01-19 14:49:47,2022-01-19 14:54:02,38 St & 30 Ave,6850.01,Crescent St & 30 Ave,6958.06,40.764175,-73.915840,40.768692,-73.924957,member
4,2177A5B57326CE9B,electric_bike,2022-01-16 14:36:06,2022-01-16 14:44:06,Pacific St & Nevins St,4362.04,Clinton St & Tillary St,4748.07,40.685376,-73.983021,40.696233,-73.991421,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893439,79E49EA8727CA662,classic_bike,2022-03-30 21:12:45,2022-03-30 21:22:15,E 72 St & York Ave,6889.12,E 58 St & 3 Ave,6762.02,40.766638,-73.953483,40.760958,-73.967245,member
1893440,9C4F6A095779AAE2,electric_bike,2022-03-21 09:33:21,2022-03-21 09:39:45,W 48 St & Rockefeller Plaza,6626.11,E 58 St & 3 Ave,6762.02,40.757769,-73.979294,40.760958,-73.967245,member
1893441,C1966A89B5078124,electric_bike,2022-03-04 14:31:36,2022-03-04 14:43:16,Central Park West & W 85 St,7354.01,E 58 St & 3 Ave,6762.02,40.784760,-73.969862,40.760958,-73.967245,member
1893442,A11D1F167EF1B8C6,classic_bike,2022-03-18 18:43:43,2022-03-18 18:53:41,W 56 St & 6 Ave,6809.07,E 58 St & 3 Ave,6762.02,40.763406,-73.977225,40.760958,-73.967245,member


In [ ]:
# three_month_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3998280 entries, 0 to 1893443
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 427.1+ MB


In [ ]:
# write to csv for further Tableau works

# three_month_df.to_csv("Resources/cleanData/three_mth_data.csv", index = False)